In [1]:
!pip install ftfy

     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=50787fb9b1a501d182eaa5fcfc4aec8dbb0f463cdd0ba0fb0f9ce084706ccd85
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy


In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [4]:
import re #matching regular expression
import ftfy #fixes text for you:fixes all the junk characters
import nltk#natural language toolkit
import itertools #for iterations to matrix
import numpy as np #for array
import warnings #overcome warnings
warnings.filterwarnings("ignore")
import pandas as pd #read data
import pickle as pkl #store tokenizer
from pathlib import Path #create json file
from nltk import PorterStemmer #for stemming process
import matplotlib.pyplot as plt #plot confusion matrix
from nltk.corpus import stopwords #for remove stop words
from sklearn.model_selection import train_test_split #split data
from keras.models import Model, Sequential #adding layers one by one
from keras.optimizers import Adam #for optimize the training
from keras.preprocessing.text import Tokenizer #to genarate unique number for uniq words
from keras.preprocessing.sequence import pad_sequences #for making all the record has same shape
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score #for evaluate the model accuracy
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D #Layers for build model

In [5]:
max_length=300
nb_max_words=100000
embedding_dim=10

In [7]:
fake_news = '/content/gdrive/MyDrive/Fake_News/Fake.csv'
true_news = '/content/gdrive/MyDrive/Fake_News/True.csv'

In [8]:
df_fake_news = pd.read_csv(fake_news)
df_true_news = pd.read_csv(true_news)

In [9]:
cList = pkl.load(open('/content/gdrive/MyDrive/Fake_News/cword_dict.pkl','rb'))

In [10]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [11]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [13]:
def clean_news(newses):
    cleaned_news = []
    for news in newses:
        news = str(news)
        if re.match("(\w+:\/\/\S+)", news) == None and len(news) > 5:
            news = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", news).split())
            news = ftfy.fix_text(news)
            news = expandContractions(news)
            news = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", news).split())
            stop_words = stopwords.words('english')
            word_tokens = nltk.word_tokenize(news) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            news = ' '.join(filtered_sentence)
            news = PorterStemmer().stem(news)
            cleaned_news.append(news)
    return cleaned_news

In [14]:
arr_fake_news = [x for x in df_fake_news['text']]
arr_true_news = [x for x in df_true_news['text']]

In [15]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
cleaned_fake_news = clean_news(arr_fake_news)
cleaned_true_news = clean_news(arr_true_news)

In [17]:
# tokenizer = Tokenizer(num_words=nb_max_words)
# print(tokenizer)
# tokenizer.fit_on_texts(cleaned_fake_news + cleaned_true_news)
from keras.preprocessing.text import one_hot

In [18]:
# sequences_fake = tokenizer.texts_to_sequences(cleaned_fake_news)
# sequences_true = tokenizer.texts_to_sequences(cleaned_true_news)

sequences_fake=[one_hot(words,nb_max_words) for words in cleaned_fake_news]
sequences_true=[one_hot(words,nb_max_words) for words in cleaned_true_news]

In [19]:
data_f = pad_sequences(sequences_fake,padding='pre', maxlen=max_length)
data_t = pad_sequences(sequences_true,padding='pre', maxlen=max_length)
print('Shape of data_fake tensor:', data_f.shape)
print('Shape of data_true tensor:', data_t.shape)

Shape of data_fake tensor: (22755, 300)
Shape of data_true tensor: (21416, 300)


In [20]:
print(data_f.shape, data_t.shape)

(22755, 300) (21416, 300)


In [21]:
data = np.concatenate((data_f, data_t))

In [22]:
labels_f = np.ones(data_f.shape[0])
labels_t = np.zeros(data_t.shape[0])
print(labels_f.shape, labels_t.shape)

(22755,) (21416,)


In [23]:
labels = np.concatenate((labels_f, labels_t))

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.2, random_state=42)

In [25]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(35336, 300) (35336,) (8835, 300) (8835,)


In [27]:
from keras.layers import Flatten

In [28]:
model=Sequential()
model.add(Embedding(nb_max_words,embedding_dim,input_length=max_length))
model.add(Dropout(0.7))

model.add(Conv1D(filters=16,kernel_size=3,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.7))
model.add(Conv1D(filters=32,kernel_size=3,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.7))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 10)           1000000   
_________________________________________________________________
dropout (Dropout)            (None, 300, 10)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 16)           496       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 149, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 149, 16)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 147, 32)           1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0

In [29]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
553/553 [==============================] - 73s 99ms/step - loss: 0.6191 - accuracy: 0.5859 - val_loss: 1.3556 - val_accuracy: 0.6783
Epoch 2/10
553/553 [==============================] - 54s 97ms/step - loss: 0.1664 - accuracy: 0.9414 - val_loss: 0.9185 - val_accuracy: 0.7477
Epoch 3/10
553/553 [==============================] - 55s 100ms/step - loss: 0.1109 - accuracy: 0.9615 - val_loss: 1.0810 - val_accuracy: 0.7759
Epoch 4/10
553/553 [==============================] - 54s 99ms/step - loss: 0.0886 - accuracy: 0.9703 - val_loss: 0.7551 - val_accuracy: 0.8028
Epoch 5/10
553/553 [==============================] - 54s 98ms/step - loss: 0.0719 - accuracy: 0.9761 - val_loss: 0.9002 - val_accuracy: 0.7799
Epoch 6/10
553/553 [==============================] - 55s 100ms/step - loss: 0.0536 - accuracy: 0.9824 - val_loss: 0.7393 - val_accuracy: 0.8264
Epoch 7/10
553/553 [==============================] - 55s 99ms/step - loss: 0.0539 - accuracy: 0.9828 - val_loss: 0.9190 - val_accurac

In [30]:
model.save('CNN-LSTM_model.h5')

In [ ]:
from keras.models import model_from_json

In [ ]:
model_json=model.to_json()
with open('CNN-LSTM_model.json','w') as json_file:
  json_file.write(model_json)

model.save_weights('CNN-LSTM_weights.h5')

NotImplementedError: ignored